In [ ]:
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics

In [ ]:
# %% [code]
kms_per_radian = 6371.0088

In [ ]:
# %% [code]
df1 = pd.read_csv('../input/geosocial-clustering-of-places-from-check-in-data/Gowalla_totalCheckins.txt', sep='\t', header=None,nrows = 35000)
df1[:3]

In [ ]:
df1.columns = ['userid','timestamp','latitude','longitude','spotid']
df1[0:3]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pltdata = pd.read_csv('../input/geosocial-clustering-of-places-from-check-in-data/Gowalla_totalCheckins.txt', sep='\t', header=None,nrows = 35000)
pltdata.columns = ['userid','timestamp','latitude','longitude','spotid']

In [ ]:
pltdata.columns

In [ ]:
BBox = ((pltdata.longitude.min(),pltdata.longitude.max(),pltdata.latitude.min(), pltdata.latitude.max()))
BBox

In [ ]:
# %% [code]
## from scipy.spatial.distance import pdist, squareform
from haversine import haversine, Unit
#distance_matrix = squareform(pdist(df, (lambda u,v: haversine(u,v))))
from sklearn.cluster import DBSCAN
import sys
data = df1[["latitude", "longitude"]]
data = data.values.astype("float32", copy = False)

In [ ]:
dbsc = DBSCAN(eps = 25, min_samples = 5).fit(data)
core_samples_mask = np.zeros_like(dbsc.labels_, dtype=bool)
core_samples_mask[dbsc.core_sample_indices_] = True
labels = dbsc.labels_

#data['c_labels'] = labels

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = data[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = data[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
# %% [code]
df1.head()

In [ ]:
# %% [code]
#latitude and longitude
#to convert numpy.ndarray to dataframe
data2 = pd.DataFrame(np.array(data),columns=['a', 'b'])
data2.head()

In [ ]:
# %% [code]
#cluster labels
data3 = pd.DataFrame(np.array(labels),columns=['c'])
data3.head()

In [ ]:
# %% [code]
#now trying to combine df1 with cluster labels
final1 = pd.concat([df1, data3], axis=1)
test1 = final1
test1

In [ ]:
# %% [code]
#test1['userid']

# %% [code]
test1['countno'] = 1
columns_to = ['countno']
test2 = test1.groupby(['userid','c'])[columns_to].count().reset_index()
test2
df3 = test2.pivot_table('countno',['userid'],'c')
df3
df4 = df3.fillna(0)
df4

In [ ]:
# %% [code]
dbscan = Model(eps=25, min_samples=3)
dbscan.fit(df4)

cluster_labels1 = dbscan.labels_
unique_labels = set(cluster_labels1)

# get the number of clusters
num_clusters = len(set(cluster_labels1))
#x =df4
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
# %% [code]
#cluster_labels1

# %% [code]
df4['u_labels'] = cluster_labels1
df4

In [ ]:
# %% [code]
df5 = df4.groupby(['u_labels']).sum().reset_index()
df5

In [ ]:
# %% [code]
dbscan = DBSCAN(eps=300, min_samples=2)
dbscan.fit(df5)

In [ ]:
#my_labels = DBSCAN(df4,eps=30,MinPts=2) 
#type(my_labels)

cluster_labels1 = dbscan.labels_
unique_labels = set(cluster_labels1)

# get the number of clusters
num_clusters = len(set(cluster_labels1))
x =df4
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
# %% [code]
df5

In [ ]:
# %% [code]
cluster_labels1

In [ ]:
# %% [code]
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
dbscan = DBSCAN(eps=30, min_samples=2)
dbscan.fit(df4)

In [ ]:
#my_labels = DBSCAN(df4,eps=30,MinPts=2) 
#type(my_labels)

In [ ]:
cluster_labels1 = dbscan.labels_
unique_labels = set(cluster_labels1)

In [ ]:
# get the number of clusters
num_clusters = len(set(cluster_labels1))
x =df4
print('Number of clusters: {}'.format(num_clusters))
df4['labels1'] = cluster_labels1
#pd.DataFrame(my_labels)
df4
# Separating out the target
y = df4.loc[:,['labels1']].values# Standardizing the features
x = StandardScaler().fit_transform(x)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df4[['labels1']]], axis = 1)
finalDf

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = unique_labels

colors = ['r', 'g', 'b']
for target in targets:
    indicesToKeep = finalDf['labels1'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
# %% [code]
df4

In [ ]:
# %% [code]
finalDf

In [ ]:
# %% [code]
colors =[]
unique_labels
cluster_labels1

In [ ]:
# %% [code]
columns_to_show = []
df5 = df4.groupby(['labels1']).sum().reset_index()
df5

In [ ]:
# %% [code]
dbscan = DBSCAN(eps=30, min_samples=1)
dbscan.fit(df5)

In [ ]:
#my_labels = DBSCAN(df4,eps=30,MinPts=2) 
#type(my_labels)

cluster_labels1 = dbscan.labels_
unique_labels = set(cluster_labels1)

In [ ]:
# get the number of clusters
num_clusters = len(set(cluster_labels1))
q =df5
print('Number of clusters: {}'.format(num_clusters))
df5['labels2']  = cluster_labels1
df5
#pd.DataFrame(my_labels)

In [ ]:
# Separating out the target
"""
p = df.loc[:,['labels2']].values# Standardizing the features
q = StandardScaler().fit_transform(q)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(q)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component p1', 'principal component  q2'])
finalDf2 = pd.concat([principalDf, df5[['labels2']]], axis = 1)
finalDf2



fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component p1', fontsize = 15)
ax.set_ylabel('Principal Component q2', fontsize = 15)
ax.set_title('3 component PCA', fontsize = 20)
targets = unique_labels

colors = ['r', 'g', 'b']
for target in targets:
    indicesToKeep = finalDf2['labels2'] == target
    ax.scatter(finalDf2.loc[indicesToKeep, 'principal component p1']
               , finalDf2.loc[indicesToKeep, 'principal component q2']
               
               , s = 50)
ax.legend(targets)
ax.grid()
"""

In [ ]:
# %% [code]
df5